# A/B Testing for ShoeFly.com
Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

If you get stuck during this project or would like to see an experienced developer work through it, click “Get Help“ to see a project walkthrough video.

## Analyzing Ad Sources

In [32]:
import pandas as pd
ad_clicks = pd.read_csv('ad_clicks.csv')

1. Examine the first few rows of ad_clicks.

In [33]:
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


2. Your manager wants to know which ad platform is getting you the most views.\
How many views (i.e., rows of the table) came from each utm_source?

In [34]:
manager_list = ad_clicks.groupby('utm_source').user_id.count().reset_index()
#print(type(ad_clicks.ad_click_timestamp[2]))
manager_list

,utm_source,user_id
0,email,255
1,facebook,504
2,google,680
3,twitter,215


3 If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed.\
Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [35]:
condition1 = lambda x: True if x == x else False
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.apply(condition1)
#ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()
ad_clicks.head()

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


4. We want to know the percent of people who clicked on ads from each utm_source.\
Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. \
Save your answer to the variable clicks_by_source.

In [36]:
clicks_by_source = ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


5. Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id.\
Save your results to the variable clicks_pivot.

In [37]:
clicks_pivot = clicks_by_source.pivot(columns='is_click', index='utm_source', values='user_id')
clicks_pivot

is_click,False,True
utm_source,,
email,175,80
facebook,324,180
google,441,239
twitter,149,66


6. Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source.\
Was there a difference in click rates for each source?

In [38]:
# clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False]) *100
def calcul(y, z):
  a=float(y+z)
  b=round(y /a *100, 2)
  return b
clicks_pivot = clicks_pivot.rename(columns={True:'yes', False:'no'})
clicks_pivot['percent_clicked'] = clicks_pivot.apply(lambda x: calcul(x.yes, x.no), axis=1)
clicks_pivot

is_click,no,yes,percent_clicked
utm_source,,,
email,175,80,31.37
facebook,324,180,35.71
google,441,239,35.15
twitter,149,66,30.70


## Analyzing an A/B Test
7. The column experimental_group tells us whether the user was shown Ad A or Ad B.\
Were approximately the same number of people shown both adds?

8. Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

9. The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.\
Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

10. For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day.

11. Compare the results for A and B. What happened over the course of the week?\
Do you recommend that your company use Ad A or Ad B?

In [39]:
experiments = ad_clicks.groupby('experimental_group').user_id.count()
#print(experiments)

experiment2 = ad_clicks.groupby(['experimental_group','is_click']).user_id.count().reset_index()
experiment2_pivot = experiment2.pivot(columns='is_click', index='experimental_group', values='user_id')
#print(experiment2_pivot)

a_clicks = ad_clicks[ad_clicks.experimental_group == 'A'].drop(['experimental_group','ad_click_timestamp'], axis=1)
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B'].drop(['experimental_group','ad_click_timestamp'], axis=1)

# 10-11
a_piv = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
#print(a_piv)
a_pivot = a_piv.pivot(columns='is_click', index='day', values='user_id')
a_pivot = a_pivot.rename(columns={True:'yes', False:'no'})
a_pivot['percent'] = a_pivot.apply(lambda x: calcul(x.yes, x.no), axis=1)

b_piv = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()
b_pivot = b_piv.pivot(columns='is_click', index='day', values='user_id')
b_pivot = b_pivot.rename(columns={True:'yes', False:'no'})
b_pivot['percent'] = b_pivot.apply(lambda x: calcul(x.yes, x.no), axis=1)

print(a_pivot)
print('\n')
print(b_pivot)

is_click       no  yes  percent
day                            
1 - Monday     70   43    38.05
2 - Tuesday    76   43    36.13
3 - Wednesday  86   38    30.65
4 - Thursday   69   47    40.52
5 - Friday     77   51    39.84
6 - Saturday   73   45    38.14
7 - Sunday     66   43    39.45


is_click       no  yes  percent
day                            
1 - Monday     81   32    28.32
2 - Tuesday    74   45    37.82
3 - Wednesday  89   35    28.23
4 - Thursday   87   29    25.00
5 - Friday     90   38    29.69
6 - Saturday   76   42    35.59
7 - Sunday     75   34    31.19
